### 트러블 슈팅
1. WebBaseLoader 로 레퍼런스가 정상적으로 인식되지 않는 문제.
 - WebBaseLoader의 한계 :  
WebBaseLoader는 간단한 텍스트 추출에 적합하며, 복잡한 HTML 구조나 JavaScript 기반의 동적 콘텐츠를 처리하지 못할 수 있다. Notion 페이지는 JavaScript를 사용해 동적으로 로드되므로, WebBaseLoader가 이를 제대로 처리하지 못했을 가능성
 - 요약된 메타데이터:
출력 내용이 주로 메타데이터(title, description, page_content)로, 실제 콘텐츠를 로드하지 못한 상태  
--> 1. Selenium이나 Playwright를 사용한 동적 렌더링  
--> 2. HTML 콘텐츠를 직접 파싱  

2. WEBaseLoader 로 레퍼런스 추가했을 때, AI 가 질문에 대한 사용자의 답변을 인식하지 못하는 문제 발생
--> 사용자의 답변을 질문과 함께 feedback_prompt 형식으로 묶어서 다시 AI에게 전달하는 방식으로 문제 해결



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

llm = ChatOpenAI(model="gpt-4o-mini")

# 동적 렌더링에 적합한 Loader로 코드수정 필요
loader = WebBaseLoader(
    web_paths=("https://teamsparta.notion.site/1-999faef604144540bd85f08f4144ccdd",
               "https://teamsparta.notion.site/2-7e07e6444f9445af844790f127d73b61",
               "https://teamsparta.notion.site/3-6cd09dfcf8a24155a5d9a09bc77c8f00",
               "https://teamsparta.notion.site/4-ANN-8c0835fc02a14bd1a0dd904641d91448",
               "https://teamsparta.notion.site/5-CNN-ebf0849b92434a70999067b90f5c407e",
               "https://teamsparta.notion.site/6-RNN-339e371a13994dacac0bd67ede7e9d92",
               "https://teamsparta.notion.site/7-Attention-99442361cf6246feb1cdd0f19d1865ee",
               "https://teamsparta.notion.site/8-NLP-de06e65f548a49f396b3e93d0659eaaa",
               "https://teamsparta.notion.site/9-ResNet-c2ba8b0f31be43eba5ce9a0fe7b000c0",
               "https://teamsparta.notion.site/10-9de6870ed0424f27bbb5049562edc6f9",
               "https://teamsparta.notion.site/11-5ee9a08839bb4cebac28b0063e421eff",
               "https://teamsparta.notion.site/12-9b1a22ad063942b9b96be163c6347280",
               "https://teamsparta.notion.site/13-b1597191907f4fc3bd72462c02839254",
               "https://teamsparta.notion.site/14-1baa68a00cf14c08806edb5a3df9df57",
               "https://teamsparta.notion.site/15-ee562a33977e4e1588ea85ac946d2b2e",
               "https://teamsparta.notion.site/16-Pytorch-bfb59c7afd624faba49eee16ab074b2b"),
)

# 로드된 문서 정상처리 여부 확인
docs = loader.load()
print(f"Loaded {len(docs)} documents.")
pprint(docs[0])

# 로드된 문서 전처리(청킹)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
print(f"Number of document chunks: {len(splits)}")

# 상위 10개의 청크 출력
print("Top 10 chunks:")
for i, chunk in enumerate(splits[:10], 1):
    print(f"\nChunk {i}:\n{chunk.page_content}")

retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([("system", """
    당신은 AI 강사입니다. 아래 context를 기반으로 하나의 퀴즈를 만들어 사용자의 대답을 기다리세요.
    퀴즈는 보기가 있는 객관식 또는 O,X 형태로 출제해주세요.
    이후, 사용자의 대답을 확인하고 아래 형식을 바탕으로 피드백을 제공하세요:
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    
    Context: {context}
    """)])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
        {"context": retriever | format_docs}
        | prompt
        | llm
        | StrOutputParser()
)

Loaded 16 documents.
Document(metadata={'source': 'https://teamsparta.notion.site/1-999faef604144540bd85f08f4144ccdd', 'title': 'Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', 'description': "A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team", 'language': 'No language found.'}, page_content='Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.')
Number of document chunks: 16
Top 10 chunks:

Chunk 1:
Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.

Chunk 2:
Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.

Chunk 3:
Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.

Chunk 4:
Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.

Chunk 5:
Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.

Chunk 6:
Notion – The all-in-one workspace

In [ ]:

# 사용자가 "exit" 을 입력할 경우, 대화가 종료됩니다.
while True: 
    query = input("질문을 입력하세요: ")
    if query.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    
    # 1. 퀴즈 생성
    quiz = rag_chain.invoke(query)
    print("Generated Quiz:")
    print(quiz)
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요: ")
    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    
    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
        AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
        퀴즈: {quiz}
        사용자의 답변: {user_answer}
        피드백:
        """)
    ])
    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "answer": user_answer})
    print("Feedback:")
    print(feedback)

